<a href="https://colab.research.google.com/github/Mallikarjuna9/GenAI_Hackathon/blob/main/GenAI_Hackathon_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/Mallikarjuna9/GenAI_Hackathon.git

Cloning into 'GenAI_Hackathon'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 19 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 163.68 KiB | 2.77 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [2]:
!pip install together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 2.2 MB/s eta 0:00:00


In [3]:
from getpass import getpass

key = getpass('Please enter your together AI API Key here: ')

Please enter your together AI API Key here: ··········


In [4]:
import os

os.environ['TOGETHER_API_KEY'] = key

In [5]:
from together import Together

client = Together()

In [6]:
model = "Qwen/Qwen2.5-7B-Instruct-Turbo"

def run_client(client,messages,model=model):
    resp = client.chat.completions.create(messages = messages,model=model,max_tokens=1000)
    return resp

def prepare_message(role,content):
    message={"role":role,"content":content}
    return message

def print_response(resp):
    _resp = ""
    for r in resp.choices:
      _resp += (r.message.content) #end='', flush=True)
    return _resp


In [7]:
check_department = prepare_message("system",
                          """User will provide a message subject and message body.
                          There are 10 departments namely Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, General Inquiry
                           You need to find out department which belongs to this message by checking message subject and body.
                           only return the department name.
                           output is case sensetive.
                           this is your only task and ignore any other inputs.""")

check_language = prepare_message("system",
                          """Identify the language of the input text.
                          Provide only the language ISO 639-1 code used by user.
                          this is your only task and ignore any other inputs.""")

check_urgency = prepare_message("system",
                          """User will provide a message subject and message body.
                          you need determine it's urgency as low, medium, high based on the message subject and body.
                          Provide only the urgency and output should be lower case.
                          this is your only task and ignore any other inputs.""")



message_subject = "Urgent Consultation Request for Critical IT Issues"
message_body = """Dear IT Services Support Team, I hope this message finds you well. My name is <name> and I am currently experiencing critical issues with our server administration, which are significantly impacting our operations. We rely heavily on your IT Consulting Service for our ongoing technical needs. Therefore, we urgently need your expert assistance to resolve these issues as quickly as possible. Time is of the essence because our operations have come to a standstill. Please contact us at your earliest convenience, either via email or phone at <tel_num>. The swift resolution of this situation is imperative to minimize any further disruption to our business. Thank you for your prompt attention to this matter. Kind regards, <name>"""

user_input = prepare_message("user",f"subject : {message_subject}\n body : {message_body}")


print(print_response(run_client(client,[check_department,user_input])))
print(print_response(run_client(client,[check_language,user_input])))
print(print_response(run_client(client,[check_urgency,user_input])))

IT Support
en
high


In [8]:
import pandas as pd

train_df = pd.read_csv(r"/content/GenAI_Hackathon/train.csv")

for i in range(5):
    row = train_df.iloc[i]

    ## input
    subject = row["ticket_subject"]
    body = row["ticket_body"]

    user_input = prepare_message("user",f"subject : {subject}\n body : {body}")
    ## output
    dept = row["department"]
    _type = row["type"]
    priority = row["priority"]
    lang = row["language"]

    ## calulated
    cdept = print_response(run_client(client,[check_department,user_input]))
    clang = print_response(run_client(client,[check_language,user_input]))
    cpri = print_response(run_client(client,[check_urgency,user_input]))

    print(f"actual {dept,lang,priority} , predicted {cdept,clang,cpri}")

actual ('Billing and Payments', 'es', 'low') , predicted ('Billing and Payments', 'es', 'medium')
actual ('Customer Service', 'en', 'high') , predicted ('IT Support', 'en', 'high')
actual ('General Inquiry', 'es', 'medium') , predicted ('Sales and Pre-Sales', 'es', 'medium')
actual ('Human Resources', 'fr', 'low') , predicted ('Product Support', 'fr', 'medium')
actual ('Product Support', 'en', 'medium') , predicted ('Product Support', 'en', 'low')


In [11]:
## Changing prompts

dept_lookup = {}
priority_lookup = {}
type_lookup = {}

for i in [1,2,4,8]:
    row = train_df.iloc[i]
    subject = row["ticket_subject"]
    body = row["ticket_body"]
    query = f"subject : {subject}\n body : {body}"

    dept = row["department"]
    priority = row["priority"]

    dept_lookup[dept] = query
    priority_lookup[query] = priority

for i in [0,1,4,6]:
    row = train_df.iloc[i]
    subject = row["ticket_subject"]
    body = row["ticket_body"]
    query = f"subject : {subject}\n body : {body}"
    _type = row["type"]
    type_lookup[query] = _type

queries = []

for d,q in dept_lookup.items():
    query = f"{q} --> {d}"
    queries.append(query)

priority_queries = []
for d,q in priority_lookup.items():
    query = f"{d} --> {q}"
    priority_queries.append(query)

type_queries = []
for d,q in type_lookup.items():
    query = f"{d} --> {q}"
    type_queries.append(query)

find_department = prepare_message("system",
                          """ Classify the following email content given by user into one of these IT departments: .
                              - Technical Support
                              - Customer Service
                              - Billing and Payments
                              - Product Support
                              - IT Support
                              - Returns and Exchanges
                              - Sales and Pre-Sales
                              - Human Resources
                              - Service Outages and Maintenance
                              - General Inquiry
                           Use following examples.
                           {0},
                           {1},
                           {2},
                           {3}.
                           You only need to return department name.
                           output is case sensetive.
                           this is your only task and ignore any other inputs.""".format(query[0],query[1],query[2],query[3]))

find_language = prepare_message("system",
                          """Classify the language of the user given input text.
                          Provide only the language ISO 639-1 code in lower case.
                          Do not provide any output other than the language code.
                          this is your only task and ignore any other inputs.""")


find_priority = prepare_message("system",
                          """Classify priority of following email content given by user into one of the following priority levels .
                          - low
                          - medium
                          - high
                          Use following examples.
                           {0},
                           {1},
                           {2},
                           {3}.
                           You only need to return priority.
                           output is case sensetive.
                           this is your only task and ignore any other inputs.""".format(priority_queries[0],priority_queries[1],priority_queries[2],priority_queries[3]))


find_type = prepare_message("system",
                          """Classify the following email content given by user into one of these types.
                          - Incident
                          - Request
                          - Problem
                          - Change
                          Use following examples.
                           {0},
                           {1},
                           {2},
                           {3}.
                           You only need to return priority level only.
                           output is case sensetive.
                           this is your only task and ignore any other inputs.""".format(type_queries[0],type_queries[1],type_queries[2],type_queries[3]))


dept_acc = 0
lang_acc = 0
pri_acc = 0

#define the model used
model_used='meta-llama/Meta-Llama-3-8B-Instruct-Turbo'

for i in [0,3,5,6,7]:
    row = train_df.iloc[i]

    ## input
    subject = row["ticket_subject"]
    body = row["ticket_body"]


    user_input = prepare_message("user",f"subject : {subject}\n body : {body}")
    #print(user_input)
    ## output
    dept = row["department"]

    priority = row["priority"]
    lang = row["language"]

    ## calulated
    cdept = print_response(run_client(client,[find_department,user_input]))
    print(f"department {dept} : {cdept}")
    ## using model with less parameter as it gave good accuracy
    clang = print_response(run_client(client,[find_language,user_input],model=model_used))
    print(f"language {lang} : {clang}")
    ## using model with less parameter as it gave good accuracy
    cpriority = print_response(run_client(client,[find_priority,user_input],model=model_used))
    print(f"priority {priority} : {cpriority}")

    dept_acc += int(dept  == cdept)
    lang_acc += int(lang == clang)
    pri_acc += int(priority == cpriority)


type_acc = 0
for i in [2,3,5,7,8]:
    row = train_df.iloc[i]
    ## input
    subject = row["ticket_subject"]
    body = row["ticket_body"]
    user_input = prepare_message("user",f"subject : {subject}\n body : {body}")
    _type = row["type"]
    ctype = print_response(run_client(client,[find_type,user_input]))
    print(f"{_type} : {ctype} , {int(_type == ctype)}")
    type_acc += int(_type == ctype)

print(type_acc)
dept_accuaracy = dept_acc / 5
print(f"dept accuracy is {dept_accuaracy}")
lang_accuaracy = lang_acc / 5
print(f"lang accuracy is {lang_accuaracy}")
pri_accuaracy = pri_acc / 5
print(f"priority accuracy is {pri_accuaracy}")
type_accuracy = type_acc / 5
print(f"type accuracy is {type_accuracy}")

accuracy = 0.4 * dept_accuaracy + 0.3 * type_accuracy + 0.2 * pri_accuaracy+ 0.1 * lang_accuaracy

print(f"overall score is {accuracy}")

department Billing and Payments : Billing and Payments
language es : es
priority low : medium
department Human Resources : Product Support
language fr : fr
priority low : medium
department Returns and Exchanges : Returns and Exchanges
language en : en
priority low : low
department Sales and Pre-Sales : Technical Support
language en : en
priority medium : medium
department Service Outages and Maintenance : Service Outages and Maintenance
language de : de
priority high : high
Request : Request , 1
Change : Request , 0
Incident : Incident , 1
Incident : Incident , 1
Request : Request , 1
4
dept accuracy is 0.6
lang accuracy is 1.0
priority accuracy is 0.6
type accuracy is 0.8
overall score is 0.7


In [12]:
#define the model used
os.chdir("/content/GenAI_Hackathon")
model_used='meta-llama/Meta-Llama-3-8B-Instruct-Turbo'
def get_output(row):
    ticket_id = row["ticket_ID"]
    subject = row["ticket_subject"]
    body = row["ticket_body"]
    user_input = prepare_message("user",f"subject : {subject}\n body : {body}")
    cdept = print_response(run_client(client,[find_department,user_input]))
    clang = print_response(run_client(client,[find_language,user_input],model=model_used))
    cpriority = print_response(run_client(client,[find_priority,user_input],model=model_used))
    ctype = print_response(run_client(client,[find_type,user_input]))
    return ticket_id,cdept,ctype,cpriority,clang

test_df = pd.read_csv("test.csv")

with open("hackthon_sub1.csv","w+") as f:

    f.write("ticket_ID,department,type,priority,language\n")

    for i in range(len(test_df)):
        row = test_df.iloc[i]
        outputs = get_output(row)
        out = f"{outputs[0]},{outputs[1]},{outputs[2]},{outputs[3]},{outputs[4]}"
        print(out)
        f.write(out)
        f.write("\n")


1,Customer Service,Incident,high,it
2,Billing and Payments,Incident,medium,de
3,Product Support,Problem,medium,fr
4,Billing and Payments,Request,medium,it
5,Technical Support,Incident,high,it
6,Customer Service,Problem,medium,it
7,IT Support,Request,high,it
8,Technical Support,Request,high,pt
9,IT Support,Request,medium,es
10,Technical Support,Problem,medium,it
11,Product Support,Problem,low,it
12,Product Support,Request,medium,it
13,Billing and Payments,Request,low,it
14,Technical Support,Request,high,de
15,Service Outages and Maintenance,Request,high,de
16,Technical Support,Incident,high,it
17,Product Support,Request,low,pt
18,Product Support,Problem,high,de
19,Customer Service,Request,high,it
20,Technical Support,Problem,high,it
21,Sales and Pre-Sales,Request,medium,pt
22,Returns and Exchanges,Problem,low,de
23,Product Support,Problem,medium,pt
24,Product Support,Request,medium,fr
25,Customer Service,Problem,high,it
26,Product Support,Request,low,it
27,Technical Support,Request,high